In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_csv, col
import random

In [3]:
scala_version = '2.12'
spark_version = '3.5.0'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}' ,
                                                'org.apache.kafka:kafka-clients:3.6.0']
spark = SparkSession.builder.master("local")\
                            .appName("Spotify Recommendation System")\
                            .config("spark.jars.packages", ",".join(packages))\
                            .getOrCreate()

23/11/24 23:19:27 WARN Utils: Your hostname, dothinh.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
23/11/24 23:19:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/Cellar/apache-spark/3.5.0/libexec/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/dothinhtpr247gmai.com/.ivy2/cache
The jars for the packages stored in: /Users/dothinhtpr247gmai.com/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-32e5e406-223d-4e8c-94c6-577085114412;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.6.0 in central
	found com.github.

## Pyspark consumer for streaming data

In [4]:
KAFKA_TOPIC_NAME_CONS = "songTopic"
KAFKA_BOOTSTRAP_SERVERS_CONS = 'localhost:9092'

# Construct a streaming DataFrame that reads from test-topic
songs_df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS_CONS) \
        .option("subscribe", KAFKA_TOPIC_NAME_CONS) \
        .option("startingOffsets", "latest") \
        .load()

songs_df1 = songs_df.selectExpr("CAST(value AS STRING)", "timestamp")


songs_schema_string = "order_id INT,id STRING, name STRING,popularity INT, duration_ms DOUBLE, explicit INT, " \
                           + "artists STRING, id_artists STRING, release_date STRING, " \
                           + "danceability DOUBLE," \
                           + "energy DOUBLE, key INT, loudness DOUBLE, " \
                           + "mode INT," \
                           + "speechiness DOUBLE," \
                           + "acousticness DOUBLE, instrumentalness DOUBLE, liveness DOUBLE, " \
                           + "valence DOUBLE, tempo DOUBLE, time_signature DOUBLE"



songs_df2 = songs_df1 \
        .select(from_csv(col("value"), songs_schema_string) \
                .alias("song"), "timestamp")


songs_df3 = songs_df2.select("song.*", "timestamp")

## Spark SQL View
songs_df3.createOrReplaceTempView("song_find");
song_find_text = spark.sql("SELECT * FROM song_find")
songs_agg_write_stream = song_find_text \
        .writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format("memory") \
        .queryName("testedTable5") \
        .start()

songs_agg_write_stream.awaitTermination(1)

23/11/24 23:19:37 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/n4/b5xh84d97qzb6sptmp370phr0000gn/T/temporary-b7e3d6f1-7569-44b6-a9a1-8e5b7a89c114. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/11/24 23:19:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


False

In [5]:
# from time import sleep
# from IPython.display import display, clear_output

# for x in range(2000):
#         try:
#                 print("Showing live view refreshed every 5 seconds")
#                 print(f"Seconds passed: {x*5}")
#                 result = spark.sql(f'SELECT * from {songs_agg_write_stream.name}')
#                 display(result.toPandas())
#                 sleep(5)
#                 clear_output(wait = True)
#         except KeyboardInterrupt:
#                 print("break")
#                 break
# print("Live view ended...")

In [6]:
from spotify_api import getSong
song_data = getSong.passs()
#song_data.rename(columns={'duration_s': 'duration_ms' }, inplace=True)
song_data = song_data.drop(['id', 'added_at', 'time_signature','duration_s'], axis='columns')
rand_n = random.randint(1,len(song_data))
add_df = song_data.head(rand_n)[-1:]

16 tracks


In [7]:
df = spark.sql(f"SELECT * FROM {songs_agg_write_stream.name}")
display(df.toPandas())
df = df.sort(df.release_date.desc())

df_stream = df

df = df.drop('order_id',
				'id',
				'explicit',
				'mode',
				'release_date',
				'id_artists',
				'time_signature',
				'duration_ms',
				'timestamp')

df_sp = spark.createDataFrame(add_df)
train_df = df.union(df_sp) 

,order_id,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,timestamp
0,13,4so0Wek9Ig1p6CRCHuINwW,White Christmas,56,177480.0,0,BingCrosbyKenDarbySingersJohnScottTrotterHisOr...,ZjFtWeHPXNFeKSUeSUfPJgUmFFAVTqGVZiLpvygCZZlPMy...,1942-01-01,0.317,...,-16.485,1,0.0381,0.673,0.353000,0.338,0.247,130.503,3.0,2023-11-24 23:19:38.773
1,14,2W889aLIKxULEefrleFBFI,Someone to Watch Over Me,54,198000.0,0,FrankSinatra,MxqyypSjfkZZLQVxS,1943,0.204,...,-17.842,1,0.0418,0.947,0.000009,0.321,0.134,91.783,3.0,2023-11-24 23:19:39.775


In [8]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=[
										'danceability',
										'energy',
										'loudness',
										'speechiness',
										'acousticness',
										'instrumentalness',
										'liveness',
										'valence',
										'tempo'
         							], 
                            outputCol='features')

assembled_data = assembler.setHandleInvalid("skip").transform(train_df)

In [9]:
from pyspark.ml.feature import StandardScaler

scale = StandardScaler(inputCol='features', outputCol='standardized')

data_scale = scale.fit(assembled_data)
df = data_scale.transform(assembled_data)

In [10]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score = []
evaluator = ClusteringEvaluator(predictionCol = 'prediction', 
                                featuresCol = 'standardized',
                                metricName = 'silhouette',
                                distanceMeasure = 'squaredEuclidean')


KMeans_algo = KMeans(featuresCol = 'standardized', k = 3)
    
KMeans_fit = KMeans_algo.fit(df)
    
output_df = KMeans_fit.transform(df)

23/11/24 23:19:49 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/24 23:19:49 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [23]:
output_df.toPandas().head()

,name,popularity,artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,features,standardized,prediction
0,What'd I Say,None,0,1959.000,0.5400,None,4.000,1.0000,0.0508,0.808000,0.0000,0.162,0.794,"[1959.0, 0.54, 4.0, 1.0, 0.0508, 0.808, 0.0, 0...","[18553.099695110464, 5.012405988277569, 0.4002...",1
1,Take Five,68,TheDaveBrubeckQuartet,0.454,0.2600,8,-13.193,0.0401,0.5390,0.000780,0.0675,0.598,174.322,"[0.454, 0.26, -13.193, 0.0401, 0.539, 0.00078,...","[4.299697428065417, 2.4133806610225332, -1.320...",1
2,Blue in Green (feat. John Coltrane & Bill Evans),65,MilesDavisJohnColtraneBillEvans,0.421,0.0162,2,-25.358,0.0388,0.8080,0.002400,0.0978,0.154,110.479,"[0.421, 0.0162, -25.358, 0.0388, 0.808, 0.0024...","[3.9871643551003095, 0.15037217964832705, -2.5...",2
3,I Only Have Eyes for You,62,TheFlamingos,0.552,0.3130,5,-12.410,0.0296,0.9160,0.001700,0.1200,0.303,88.554,"[0.552, 0.313, -12.41, 0.0296, 0.916, 0.0017, ...","[5.227825947779978, 2.9053390265386647, -1.241...",1
4,Johnny B. Goode,77,ChuckBerry,0.534,0.8030,10,-9.129,0.0743,0.7410,0.000061,0.3070,0.969,167.983,"[0.534, 0.803, -9.129, 0.0743, 0.741, 6.07e-05...","[5.057353362526284, 7.453633349234978, -0.9134...",1


23/11/25 00:11:10 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 0 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/11/25 00:11:10 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 0 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/11/25 00:11:10 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 0 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/11/25 00:11:11 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 0 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/11/25 00:11:12 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 0 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/11/25 00:11:13 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 0 (localhost/127

In [11]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

class SpotifyRecommender():
    def __init__(self, rec_data):
        self.rec_data_ = rec_data
    
    def spotify_recommendations(self, song_name, amount=1):
        distances = []
        song = self.rec_data_[(self.rec_data_.name.str.lower() == song_name.lower())]\
                                                                    .head(1).values[0]
        # get details of our fav song from name we pass as x earlier.
        res_data = self.rec_data_[self.rec_data_.name.str.lower() != song_name.lower()]
        #dropping the data with our fav song so that it doesnt affect our recommendation.
        for r_song in tqdm(res_data.values):
            # tqdm is just used for showing the bar of iteration through our streamed songs.
            dist = 0
            for col in np.arange(len(res_data.columns)):
                # (len(res_data.columns) gets us the number of columns -> 13 in our case.
                #indeces of non-numerical columns neednt be considered.
                if not col in [0,1,13]:
                    #calculating the manhettan distances for each numerical feature
                    # song -> from our fav dataset.
                    # r_song -> from streaming data.
                    dist = dist + np.absolute(float(song[col]) - float(r_song[col]))
            distances.append(dist)
            # distances are calculated and appended and added to a new column called distances in our dataset.
        res_data['distance'] = distances
        #sorting our data to be ascending by 'distance' feature
        res_data = res_data.sort_values('distance')
        # resulting dataset have the song similar to our fav song's numerical values and thus recommended.
        columns = ['name', 'artists', 
                   'acousticness', 'liveness', 
                   'instrumentalness', 'energy', 
                   'danceability', 'valence']
        
        return res_data[columns][:amount]
    
    
datad = output_df.select('name',
                        'artists',
                        'danceability',
                        'energy',
                        'key',
                        'loudness',
                        'speechiness',
                        'acousticness',
                        'instrumentalness',
                        'liveness',
                        'valence',
                        'tempo',
                        'prediction')



datf = datad.toPandas()
datf.drop(datf[datf['artists'] == '0'].index, inplace = True)
datf.drop_duplicates(inplace=True)
datf.drop(datf[datf['danceability'] == 0.0000].index, inplace = True)
datf.drop(datf[datf['liveness'] == 0.000].index, inplace = True)
datf.drop(datf[datf['instrumentalness'] == 0.000000].index, inplace = True)
datf.drop(datf[datf['energy'] == 0.0000].index, inplace = True)
datf.drop(datf[datf['danceability'] == 0.000].index, inplace = True)
datf.drop(datf[datf['valence'] == 0.000].index, inplace = True)

y = datf
value_pred = datf.iloc[-1:]['prediction']
#datf = datf[datf['prediction'] == list(value_pred)[0]]

recommender = SpotifyRecommender(datf)
x = add_df['name'].tolist()[0]

rec_song = recommender.spotify_recommendations(x, 10)

v = add_df[['name', 'artists',  'acousticness', 'liveness', 'instrumentalness', 'energy', 
       'danceability', 'valence']]

rec_song = pd.concat([rec_song, v])
rec_song.to_csv('rec_song.csv')

100%|██████████| 10/10 [00:00<00:00, 29599.89it/s]                              


In [12]:
df_rec = spark.createDataFrame(rec_song)
df_rec.show()

+--------------------+--------------------+------------+--------+----------------+------+------------+-------+
|                name|             artists|acousticness|liveness|instrumentalness|energy|danceability|valence|
+--------------------+--------------------+------------+--------+----------------+------+------------+-------+
|Again (with The M...| DorisDayTheMellomen|       0.981|  0.0753|         1.13E-6|0.0209|       0.495|  0.145|
|I Can't Quit You ...|         WillieDixon|       0.407|   0.215|          0.0434| 0.236|       0.469|  0.415|
|    Gymnopédie No. 1|ErikSatiePhilippe...|       0.994|  0.0941|           0.937|0.0128|       0.469|  0.354|
|Someone to Watch ...|        FrankSinatra|       0.947|   0.321|         9.15E-6| 0.151|       0.204|  0.134|
|Put Your Dreams A...|        FrankSinatra|        0.95|   0.152|           0.276|0.0546|       0.197|    0.1|
|Contigo - Tema Re...|          LosPanchos|       0.979|   0.102|         8.53E-4| 0.297|       0.789|   0.73|
|